## The imports

In [1]:
import zarr
import xarray as xr
import time
import dask
from intake import open_catalog


# The workers

In [2]:
nb_workers=50

In [3]:
from dask.distributed import Client
from dask_gateway import Gateway

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.scale(nb_workers)
c = Client(cluster)
c

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.a48e4a93cc664e92a8f6554112c8e618 Dashboard: https://us-central1-b.gcp.pangeo.io/services/dask-gateway/clusters/prod.a48e4a93cc664e92a8f6554112c8e618/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [10]:
cluster.scale(60)

In [4]:
from dask.utils import ensure_dict, format_bytes
workers = c.scheduler_info()["workers"]
text="Workers= " + str(len(workers))
memory = [w["memory_limit"] for w in workers.values()]
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


Workers= 50, Memory=214.75 GB


# The data

In [5]:
%%time 

import xarray as xr
import gcsfs

fs = gcsfs.GCSFileSystem('pangeo-181919', requester_pays=True)

zmap = fs.get_mapper('pangeo-meom/autre-eNATL60-BLBT02-SSH-1h/eNATL60-BLBT02-SSH-1h')
ds = xr.open_zarr(zmap)


CPU times: user 803 ms, sys: 197 ms, total: 1 s
Wall time: 6.59 s


In [6]:
ds

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, time_counter: 11688, x: 8354, y: 4729)
Coordinates:
    nav_lat               (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
    nav_lon               (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
    time_centered         (time_counter) datetime64[ns] dask.array<chunksize=(11688,), meta=np.ndarray>
  * time_counter          (time_counter) datetime64[ns] 2009-06-30T00:30:00 ... 2010-10-29T23:30:00
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    sossheig              (time_counter, y, x) float32 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
    time_centered_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(11688, 2), meta=np.ndarray>
    time_counter_bounds   (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(11688, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    TimeStamp:    08/01/2019 09:34:21 +0100
    description:  ocean T grid variables
    file_name:    eNATL60-BLBT02_1h_20090630_20090704_gridT-2D_20090630-20090...
    ibegin:       0
    jbegin:       0
    name:         /scratch/tmp/3746956/eNATL60-BLBT02_1h_20090630_20090704_gr...
    ni:           8354
    nj:           9
    timeStamp:    2018-Dec-31 17:48:48 GMT
    title:        ocean T grid variables
    uuid:         bd4ccb1a-e144-450d-b3d7-330382974e7a

In [7]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 213 ms, sys: 6.01 ms, total: 219 ms
Wall time: 215 ms


In [8]:
mean

<xarray.DataArray 'sossheig' (y: 4729, x: 8354)>
dask.array<mean_agg-aggregate, shape=(4729, 8354), dtype=float32, chunksize=(4729, 8354), chunktype=numpy.ndarray>
Coordinates:
    nav_lat  (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
    nav_lon  (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
Dimensions without coordinates: y, x

In [11]:
%time mean.load()

KilledWorker: ("('zarr-12ec307444d832d836bd0c4818df44ca', 1789, 0, 0)", <Worker 'tls://10.36.137.6:40447', name: dask-gateway-dvfj4, memory: 0, processing: 273>)

In [11]:
%time mean_space=ds.sossheig.mean(dim={'x','y'})

CPU times: user 473 ms, sys: 37 µs, total: 473 ms
Wall time: 481 ms


In [12]:
mean_space

,Array,Chunk
Bytes,46.75 kB,4 B
Shape,"(11688,)","(1,)"
Count,35065 Tasks,11688 Chunks
Type,float32,numpy.ndarray


In [13]:
%time mean_space.load()

CPU times: user 14.8 s, sys: 1.65 s, total: 16.5 s
Wall time: 6min 45s


<xarray.DataArray 'sossheig' (time_counter: 11688)>
array([-0.2624576 , -0.26359338, -0.2584397 , ..., -0.22629748,
       -0.24492481, -0.25286493], dtype=float32)
Coordinates:
    time_centered  (time_counter) datetime64[ns] 2009-06-30T00:30:00 ... 2010-10-29T23:30:00
  * time_counter   (time_counter) datetime64[ns] 2009-06-30T00:30:00 ... 2010-10-29T23:30:00

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/tornado/iostream.py", line 1417, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.7/ssl.py", line 1139, in do_handshake
    self._sslobj.do_handshake()
OSError: [Errno 0] Error

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/tornado/iostream.py", line 711, in _handle_events
    self._handle_read()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/tornado/iostream.py", line 1498, in _handle_read
    self._do_ssl_handshak